In [ ]:
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy
import tensorflow as tf
from tensorflow.python.framework import ops

In [ ]:
# read files, specity dtype to be short float for float columns
train_sample = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/train.csv',                
                    converters={'unit_sales': lambda u: float(u) if float(u) > 0 else 0})
stores = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/stores.csv')
items = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/items.csv')
transactions = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/transactions.csv', 
                          dtype = {'transactions': 'int32'})
oil = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/oil.csv', names = ['date', 'oil_price'], header=0,
                 dtype = {'oil_price': 'float32'})

In [ ]:
print ('dimention of train_sample = ', train_sample.shape)
print ('dimention of stores = ', stores.shape)
print ('dimention of items = ', items.shape)
print ('dimention of oil = ', oil.shape)
print ('dimention of transactions = ', transactions.shape)

print ('dtypes of train_sample = ', train_sample.dtypes)
print ('dtypes of stores = ', stores.dtypes)
print ('dtypes of items = ', items.dtypes)
print ('dtypes of oil = ', oil.dtypes)
print ('dtypes of transactions = ', transactions.dtypes)



In [ ]:
#extract features for the particular date(oil price, stores (e.g. locations and transactions)
#and items (e.g. types) 
join1 = train_sample.join(oil.set_index('date'), on = 'date')
join2 = join1.join(stores.set_index('store_nbr'), on = 'store_nbr', rsuffix = '_store')
join3 = join2.join(items.set_index('item_nbr'), on = 'item_nbr', rsuffix = '_item')
train = join3.join(transactions.set_index(['date','store_nbr']), on = ['date', 'store_nbr'], rsuffix = '_store')

#Sort out holiday information
holidays = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/holidays_events.csv')
#If a holiday is transferred it is not celebrated
holidays.loc[holidays['transferred'] == True, 'holiday_event'] = False
holidays.loc[holidays['transferred'] != True, 'holiday_event'] = True
#Work day is meant to make up "bridage'
holidays.loc[holidays['type'] == 'Work Day', 'holiday_event'] = False

train = train.join(holidays.set_index('date'), on = 'date', rsuffix = '_holidays')
train['holiday_loc_spec'] = False
train['holiday_event'] = train['holiday_event'].astype('bool')

# Step1: if it is National holiday, holiday_loc_spec is true 
train.loc[(train['holiday_event'] == True) & (train['locale'] == 'National'), 'holiday_loc_spec'] = True

# Step2: if it is Regional holiday (state == locale_name), holiday_loc_spec is true 
train.loc[(train['holiday_event'] == True) & (train['locale'] == 'Regional') & (train['state'] == train['locale_name']), 
          'holiday_loc_spec'] = True

# step3: if it is Local (locale_name == city), holiday_loc_spec is true 
train.loc[(train['holiday_event'] == True) & (train['locale'] == 'Local') & (train['city'] == train['locale_name']), 
     'holiday_loc_spec'] = True

#print ('holidays: ', train.loc[train['holiday_loc_spec'] == True].shape[0])
#print ('none-holidays: ', train.loc[train['holiday_loc_spec'] == False].shape[0])
# Step 4: delete some columns (metadata about holidays)
train = train.drop(columns= ['type_holidays', 'locale', 'locale_name', 'description', 'transferred'])
# get date information (parsing date manually)
train['year'] = pd.DatetimeIndex(train['date']).year.astype('object')
train['month'] = pd.DatetimeIndex(train['date']).month.astype('object')
train['dayofweek'] = pd.DatetimeIndex(train['date']).dayofweek.astype('object')

#log transform the Y
train['unit_sales_log'] = np.log(train['unit_sales'] + 1)

#hand label onpromotions missing data as not-onpromotion

train.loc[train.onpromotion.isnull(), 'onpromotion'] = False
#train['onpromotion'] = train['onpromotion'].astype(bool)

#fix dtypes so dummies variables can be generated
train['perishable'] = train['perishable'].astype('bool')
train['cluster'] = train['cluster'].astype('object')
train['class'] = train['class'].astype('object')

train_none_cate = train.loc[:, ['unit_sales_log', 'oil_price',
                                'perishable','transactions', 'holiday_loc_spec', 'onpromotion']]
train_cate = train.loc[:,['city', 'state', 'type', 'cluster', 'family', 'class', 'year', 'month', 'dayofweek']]
train_cate = pd.get_dummies(train_cate, prefix = ['city', 'state', 'type', 
                                                  'cluster', 'family', 'class', 'year', 'month', 'dayofweek'])
train = pd.concat([train_none_cate, train_cate], axis = 1)
print (train.dtypes)


In [ ]:
train.to_csv('data_preprocessed.csv')

In [ ]:
from sklearn.preprocessing import Imputer, StandardScaler
X = train.loc[:, 'oil_price':]
y = train.loc[:, 'unit_sales_log']

print ('infomation about missing value before imputing')
print (X.isnull().sum().sum())
X.fillna(X.mean(), inplace=True)

std = StandardScaler()
std.fit(X.loc[:,['oil_price']].values)
X['oil_price'] = std.transform(X.loc[:,['oil_price']].values)

std = StandardScaler()
std.fit(X.loc[:,['transactions']].values)
X['transactions'] = std.transform(X.loc[:,['transactions']].values)
print ('infomation about missing value after imputing')
print (X.isnull().sum().sum())

In [ ]:
print (X['oil_price'].describe())
print (X['transactions'].describe())
print (X.dtypes)

In [ ]:
import sklearn
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

model1 = SVR(kernel='linear', C=1)
model2 = DecisionTreeRegressor(max_depth = 2)

model1.fit(X_train, y_train)
y_hat1_train = model1.predict(X_train)
y_hat1_test = model1.predict(X_test)

model2.fit(X_train, y_train)
y_hat2_train = model2.predict(X_train)
y_hat2_test = model1.predict(X_test)

MSLE1_train = mean_squared_log_error(y_train, y_hat1_train)
MSLE2_train = mean_squared_log_error(y_train, y_hat2_train)
MSLE1_test = mean_squared_log_error(y_test, y_hat1_test)
MSLE2_test = mean_squared_log_error(y_test, y_hat2_test)
print ('train MSE, eva MSE, train MSE, eva MSE is ' , [MSLE1_train, MSLE2_train, MSLE1_train, MSLE2_test])

In [ ]:
# work with holiday file, the goal is to determined if a particuar city of a store on a particular date
# was celebrating holiday_event 
holidays = pd.read_csv('/home/sophia/Downloads/grocery_dataFiles/holidays_events.csv')
list1 = list(holidays.locale_name.unique())
list2 = list(stores.city.unique())
print ('holiday locale names:', sorted(list1))
print ('store city names:', sorted(list2))
print ('states name: ', sorted(list(stores.state.unique())))
n, bins, patches = plt.hist(holidays['type'], 50, normed=1, facecolor='green', alpha=0.75)
plt.show()


# Step one: if holiday is transferred (only holiday has transferred examples, it was not celebrated, so labeled as not holiday_event
# Step two: if holiday is not transferred, it remains as holiday_event
# step three: Work Day are meant to make up Bridge, so not celebrated as holiday_event (it is all national) 
holidays.loc[holidays['transferred'] == True, 'holiday_event'] = False
holidays.loc[holidays['transferred'] != True, 'holiday_event'] = True
holidays.loc[holidays['type'] == 'Work Day', 'holiday_event'] = False
holidays[holidays['type'] == 'Work Day']
print (holidays.dtypes)

In [ ]:
#now check the two things we have marked: holiday that has been transferred and work day that meant to make 
#up Bridge is labeled not holiday_event
#plt.style.use('seaborn-white')
plt.style.use('dark_background')
holiday_locale_type = holidays.groupby(['type', 'holiday_event']).size()
holiday_locale_type.unstack().plot(kind='bar',stacked=True, figsize=(12,10),  grid=False)
plt.title('Stacked Barplot of holiday_event label against event type')
plt.ylabel('Count of entries')
plt.show()
